## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"


### Задание 1
Парсинг новостей с сайта https://habr.com/ru/all

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta
import time

KEYWORDS = ['время', 'код']

def contains_keywords(text):
    for keyword in KEYWORDS:
        if re.findall(r'\b{}\b'.format(keyword.lower()), text.lower()):
            return True
    return False

def get_full_text(link):
    response = requests.get(link)
    time.sleep(0.5)

    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.find('div', class_='post__body post__body_full').text.strip()

def find_articles(full_version=False):
    response = requests.get('https://habr.com/ru/all/')
    time.sleep(0.5)

    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('article', class_='post post_preview')
    
    result = pd.DataFrame()
    for article in articles:
        date = article.find('span', class_='post__time').text.split('в')[0]        
        date = date.replace('сегодня', datetime.strftime(datetime.now(), '%d %B %Y'))
        date = date.replace('вчера', datetime.strftime(datetime.now() + timedelta(days=-1), '%d %B %Y'))
        
        title = article.find('h2', class_='post__title').text.strip()
        link = article.find('a', class_='post__title_link').get('href')

        if full_version:
            full_text = get_full_text(link)
            if contains_keywords(title + ' ' + full_text):
                row = {'date': date, 'title': title, 'link': link, 'text': full_text}
                result = pd.concat([result, pd.DataFrame([row])])
        else:
            short_text = article.find('div', class_='post__body post__body_crop').text.strip()
            if contains_keywords(title + ' ' + short_text):
                row = {'date': date, 'title': title, 'link': link}
                result = pd.concat([result, pd.DataFrame([row])])
    
    return result

In [2]:
find_articles() # основное задание

,date,title,link
0,02 April 2021,10 плат для начала разработки IoT в 2021,https://habr.com/ru/company/otus/blog/550446/
0,02 April 2021,Дайджест релизов мобильной разработки Mail.ru ...,https://habr.com/ru/company/mailru/blog/550384/
0,02 April 2021,"Как я нашел баг, который раскрывал ваш пароль ...",https://habr.com/ru/company/skillfactory/blog/...


In [3]:
find_articles(full_version=True) # дополнительное задание

,date,title,link,text
0,02 April 2021,10 плат для начала разработки IoT в 2021,https://habr.com/ru/company/otus/blog/550446/,Есть масса хороших вариантов для стартаИнтерне...
0,02 April 2021,Пишем свою навигацию в Qt,https://habr.com/ru/post/550440/,Всем привет. Я android разработчик с небольшим...
0,02 April 2021,"Презентация компании Airbyte, благодаря которо...",https://habr.com/ru/post/550282/,Всем привет. Я занимаюсь презентациями и иллюс...
0,02 April 2021,Мультиклеточная архитектура: тесты и развитие,https://habr.com/ru/post/550434/,В этой статье мы поговорим о развитии архитект...
0,02 April 2021,Python &amp;amp; EEL. Делаем просто на Python’...,https://habr.com/ru/post/550426/,ВступлениеТема красивых интерфейсов была и буд...
0,02 April 2021,Кто любит возмущаться,https://habr.com/ru/company/itelma/blog/550418/,В 2020 году возмущение стало новейшим наркотик...
0,02 April 2021,Дайджест релизов мобильной разработки Mail.ru ...,https://habr.com/ru/company/mailru/blog/550384/,В прошлом году мы дружно перешли на удалёнку и...
0,02 April 2021,"Как я нашел баг, который раскрывал ваш пароль ...",https://habr.com/ru/company/skillfactory/blog/...,В охоте на проблемы безопасности погоня за неи...
0,02 April 2021,Перевод статьи «В защиту Ричарда Столлмана»,https://habr.com/ru/post/550406/,"Два года назад, в результате клеветнической ка..."
0,02 April 2021,"Компьютеры, какими я их любил",https://habr.com/ru/company/itelma/blog/550404/,Я много лет боролся с проблемой синхронизации ...


### Задание 2 - основное
Have I been hacked?

In [5]:
import json

EMAIL = ["xxx@ya.ru", "yyy@gmail.com"]

headers = {
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}
json_ = json.dumps({'emailAddresses': EMAIL})

response = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data', data=json_, headers=headers)
time.sleep(0.5)

dict_ = json.loads(response.text)

breach_details = pd.DataFrame()
for email, breaches_dict in dict_['summary'].items():
    for breach in breaches_dict['breaches']:
        date = datetime.strptime(dict_['breaches'][str(breach)]['publishDate'], '%Y-%m-%dT%H:%M:%SZ').date()
        source = dict_['breaches'][str(breach)]['site']
        desc = dict_['breaches'][str(breach)]['description']

        row = {'email': email, 'date': date, 'source': source, 'desc': desc}
        breach_details = breach_details.append(row, ignore_index=True)

breach_details

,date,desc,email,source
0,2016-11-07,"In October 2015, the Chinese internet and gami...",xxx@ya.ru,ir.netease.com
1,2020-07-30,"In March 2019, the Brazilian travel agency Her...",xxx@ya.ru,hurb.com
2,2019-03-28,Big data e-mail verification platform verifica...,xxx@ya.ru,verifications.io
3,2020-05-21,"At some time in 2020, the Russian social netwo...",xxx@ya.ru,vk.com
4,2016-11-01,"In 2011, Russian instant messaging service pro...",xxx@ya.ru,qip.ru
...,...,...,...,...
135,2017-01-16,"In September 2016, CatchItEnglish.com was brea...",yyy@gmail.com,catchitenglish.com
136,2017-10-29,Online recruitment site JobStreet was allegedl...,yyy@gmail.com,jobstreet.com
137,2017-05-26,"In February 2016, LEET's user database was all...",yyy@gmail.com,leet.cc
138,2016-12-05,At an unknown date ClassroomCash.org was hacke...,yyy@gmail.com,classroomcash.org


### Задание 2 - дополнительное
50 постов со стены группы в vk

In [29]:
with open('token.txt') as f:
    TOKEN = f.readline()

GROUP = 'netology'

params = {
    'domain': GROUP,
    'count': 50,
    'filter': 'owner',
    'access_token': TOKEN,
    'v': '5.130'
}

response = requests.get('https://api.vk.com/method/wall.get', params=params)
dict_ = json.loads(response.text)

posts = pd.DataFrame()

for post in dict_['response']['items']:
    date = datetime.fromtimestamp(post['date']).date()
    text = post['text']
    posts = posts.append({'date': date, 'text': text}, ignore_index=True)

posts

,date,text
0,2021-04-02,"Пишем коды для веб-страниц, осваиваем языки пр..."
1,2021-04-01,Прокрастинация — это когда нужно работать здес...
2,2021-04-01,"Чтобы сменить специальность, нужно признать, ч..."
3,2021-03-31,Разыгрываем 10 доступов к библиотеке видеокурс...
4,2021-03-31,"Кажется, в SMM всё просто: написал пост, ответ..."
5,2021-03-30,В 2020 году сербская художница-перформанистка ...
6,2021-03-30,В новом выпуске НетоNews рассказали про самые ...
7,2021-03-29,❌ Бросить попытки после первой неудачи \n❓Учит...
8,2021-03-29,Записывайтесь на бесплатный курс «Диджитал-ста...
9,2021-03-28,Вместе с сервисом Bookmate запустили квиз по к...
